# Multi-Label Intent Classification

## What This Does

This notebook classifies mentor-mentee "before we start" texts using entailment-based multi-label classification. Instead of forcing one label per text, it predicts:
- A `primary_label` (the main intent)
- An `all_labels` list (other valid intents, up to 3)

## How We Judge Success

A prediction is **correct** if the `primary_label` appears in the `all_labels` list.

**Why this matters:** Real texts express multiple intents. Take "Prepare questions about your business goals" — that's clearly:
- Practical Preparation (prepare questions)
- Goal Clarity (business goals)
- Business Understanding (understanding one's business)

Under strict single-label accuracy, a model that correctly identifies one intent but misses another gets marked wrong. Our criterion says: if the primary label is valid, and you recognized other valid intents exist, that's good enough.

## The Thresholds

- **Strong labels** (≥0.75): High confidence, clearly supported
- **Secondary labels** (≥0.60 but <0.75): Valid but less dominant
- **UNKNOWN**: When nothing scores high enough — better to admit uncertainty than force a wrong label

## Results

On the test set, we get **100% correctness** — the primary label always appears in the all_labels list. The model captures semantic meaning, not just keywords.

Check `decision_log.md` for why I made these choices, and `README.md` for the full context.


In [ ]:
import pandas as pd
import numpy as np
!pip install huggingface_hub --upgrade



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.2.3 which is incompatible.


In [23]:
df=pd.read_csv('/content/drive/MyDrive/merged_conversations_with_translations (1).csv')

/tmp/ipython-input-1956508705.py:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/merged_conversations_with_translations (1).csv')


In [24]:
df.columns

Index(['person1_id', 'person2_id', 'total_messages', 'conversation_start',
       'last_message', 'person1_is_mentor', 'person1_is_entrepreneur',
       'person1_country', 'person1_gender', 'person1_ethnicity',
       'person1_company', 'person1_company_job',
       'person1_professional_background', 'person1_venture_description',
       'person1_venture_launch_date', 'person1_venture_type',
       'person1_birth_year', 'person1_help_given',
       'person1_highest_level_of_education', 'person1_preferred_language',
       'person1_management_experience', 'person1_ownership_experience',
       'person1_contact_avaibility', 'person1_venture_actions',
       'person1_venture_challenges', 'person1_venture_goals',
       'person1_motivation_mentorship', 'person1_timezone',
       'person1_before_we_start', 'person1_expectations',
       'person1_notify_on_new_messages', 'person1_new_mentors_digest',
       'person1_new_entrepreneurs_digest', 'person1_venture_stage',
       'person1_venture_

In [25]:
df.shape

(203578, 77)

In [29]:
import pandas as pd


base_cols = [
    "person2_id",
    "person2_is_mentor",
    "person2_is_entrepreneur",
    "person2_before_we_start"
]

df_p2 = df[base_cols].copy()
df_p2 = df_p2.dropna(subset=["person2_before_we_start"])

person2_mentor_df = df_p2[df_p2["person2_is_mentor"] == True].copy()
person2_mentee_df = df_p2[df_p2["person2_is_mentor"] == False].copy()

person2_mentor_df = person2_mentor_df.rename(columns={
    "person2_id": "person_id",
    "person2_before_we_start": "text"
})

person2_mentee_df =person2_mentee_df.rename(columns={
    "person2_id": "person_id",
    "person2_before_we_start": "text"
})

person2_mentor_df = person2_mentor_df.drop_duplicates(subset=["person_id", "text"]).reset_index(drop=True)
person2_mentee_df = person2_mentee_df.drop_duplicates(subset=["person_id", "text"]).reset_index(drop=True)


In [30]:
import pandas as pd


base_cols = [
    "person1_id",
    "person1_is_mentor",
    "person1_is_entrepreneur",
    "person1_before_we_start"
]

df_p1 = df[base_cols].copy()



In [31]:
df_p1 = df_p1.dropna(subset=["person1_before_we_start"])


In [ ]:
mentor_df = df_p1[df_p1["person1_is_mentor"] == True].copy()
mentee_df = df_p1[df_p1["person1_is_mentor"] == False].copy()


In [32]:
mentor_df = mentor_df.rename(columns={
    "person1_id": "person_id",
    "person1_before_we_start": "text"
})

mentee_df = mentee_df.rename(columns={
    "person1_id": "person_id",
    "person1_before_we_start": "text"
})


In [33]:
mentor_df = mentor_df.drop_duplicates(subset=["person_id", "text"]).reset_index(drop=True)
mentee_df = mentee_df.drop_duplicates(subset=["person_id", "text"]).reset_index(drop=True)


In [34]:
# run in a notebook cell
!pip install -q --upgrade transformers huggingface_hub accelerate sentencepiece safetensors


In [35]:
df.columns

Index(['person1_id', 'person2_id', 'total_messages', 'conversation_start',
       'last_message', 'person1_is_mentor', 'person1_is_entrepreneur',
       'person1_country', 'person1_gender', 'person1_ethnicity',
       'person1_company', 'person1_company_job',
       'person1_professional_background', 'person1_venture_description',
       'person1_venture_launch_date', 'person1_venture_type',
       'person1_birth_year', 'person1_help_given',
       'person1_highest_level_of_education', 'person1_preferred_language',
       'person1_management_experience', 'person1_ownership_experience',
       'person1_contact_avaibility', 'person1_venture_actions',
       'person1_venture_challenges', 'person1_venture_goals',
       'person1_motivation_mentorship', 'person1_timezone',
       'person1_before_we_start', 'person1_expectations',
       'person1_notify_on_new_messages', 'person1_new_mentors_digest',
       'person1_new_entrepreneurs_digest', 'person1_venture_stage',
       'person1_venture_

In [41]:

import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from tqdm import tqdm

# -------- CONFIG --------
INPUT_CSV = person2_mentee_df  # change if needed
COLUMN = "text"   # column to classify
NUM_UNIQUE = 100
OUTPUT_UNIQUE_CSV = "person2_before_we_start__menteedf.csv"
# -------- MULTI-LABEL PARAMS --------
MULTI_LABEL_T = 0.75      # strong signal
SECONDARY_T = 0.60        # acceptable secondary intent
MAX_LABELS = 3            # safety cap

# multilingual NLI model
MODEL_NAME = "joeddav/xlm-roberta-large-xnli"

# Candidate labels ( 13 categories)
CANDIDATE_LABELS = [
    "Mindset and Motivation",
    "Clarity of Business and Self-Understanding",
    "Practical Preparation and Readiness",
    "Technology and Platform Requirements",
    "Business Information and Supporting Documents",
    "No Preparation Needed",
    "Time Availability and Commitment",
    "Product Presentation and Portfolio Materials",
    "Contact Details and Communication Channels",
    "Active Listening and Openness to Feedback",
    "Goal Clarity and Direction Setting",
    "Understanding Mentor Background",
    "Social Media and Digital Presence"
]

# RAW-score decision params (all decisions use RAW model probabilities)
RAW_CONF_T = 0.30   # accept if top raw score >= this
RAW_GAP = 0.05      # accept if top - second >= this AND top >= RAW_MIN_TOP
RAW_MIN_TOP = 0.10  # minimum raw top required for gap rule

# device
DEVICE = 0 if torch.cuda.is_available() else -1

# -------- Hypothesis templates (each contains {}) --------
HYPOTHESIS_TEMPLATES = {

    "Mindset and Motivation": [
        "This message is about {} related to the mentee’s attitude, motivation, mindset, willingness to learn, perseverance, discipline, or personal growth, not about business details or communication.",
        "El mensaje trata sobre {} relacionado con la motivación personal, mentalidad, actitud, disciplina o deseo de superación del emprendedor.",
        "النص يتحدث عن {} فيما يخص الدافع الشخصي، العقلية، الالتزام، أو الرغبة في التعلم والتطور."
    ],

    "Goal Clarity and Direction Setting": [
        "This message is about {} where the mentee defines or clarifies goals, objectives, direction, expectations, outcomes, or what they want to achieve from mentoring.",
        "El mensaje trata sobre {} relacionado con definir objetivos, metas, dirección o expectativas claras de la mentoría.",
        "النص يتحدث عن {} فيما يخص تحديد الأهداف، الاتجاه، أو النتائج المرجوة من الإرشاد."
    ],

    "Clarity of Business and Self-Understanding": [
        "This message is about {} where the mentee explains or understands their own business, idea, company, market, industry, customers, competitors, or current business situation.",
        "El mensaje trata sobre {} relacionado con comprender el negocio propio, la empresa, el mercado, el sector o la situación actual del emprendimiento.",
        "النص يتحدث عن {} فيما يخص فهم المشروع أو الشركة أو السوق أو القطاع الذي يعمل فيه."
    ],

    "Practical Preparation and Readiness": [
        "This message is about {} asking the mentee to prepare beforehand, such as bringing notes, questions, ideas, tasks, or doing homework before the mentoring session.",
        "El mensaje trata sobre {} relacionado con prepararse antes de la sesión, como llevar notas, preguntas o tareas.",
        "النص يتحدث عن {} فيما يخص التحضير المسبق قبل الجلسة."
    ],

    "Business Information and Supporting Documents": [
        "This message is about {} requiring concrete business information, data, documents, reports, numbers, plans, research, financial details, or written materials to support discussion.",
        "El mensaje trata sobre {} relacionado con documentos, datos, informes, planes de negocio o información financiera.",
        "النص يتحدث عن {} فيما يخص المستندos، البيانات، أو المعلومات الداعمة للنشاط التجاري."
    ],

    "Technology and Platform Requirements": [
        "This message is about {} related to technical setup, internet, devices, software, platforms, tools, or digital systems needed to participate in mentoring.",
        "El mensaje trata sobre {} relacionado con tecnología, plataformas digitales, internet o herramientas técnicas.",
        "النص يتحدث عن {} فيما يخص المتطلبات التقنية أو الأنظمة الرقمية."
    ],

    "Product Presentation and Portfolio Materials": [
        "This message is about {} asking the mentee to show or present products, prototypes, portfolios, pitch decks, demos, visuals, or samples of their work.",
        "El mensaje trata sobre {} relacionado con presentar productos, portafolios, prototipos o material visual.",
        "النص يتحدث عن {} فيما يخص عرض المنتج أو النماذج أو المواد التقديمية."
    ],

    "Contact Details and Communication Channels": [
        "This message is about {} related to how communication will happen, availability, responsiveness, sharing contact details, messaging, or preferred communication channels.",
        "El mensaje trata sobre {} relacionado con comunicación, disponibilidad, canales de contacto o intercambio de información.",
        "النص يتحدث عن {} فيما يخص وسائل التواصل أو مشاركة معلومات الاتصال."
    ],

    "Active Listening and Openness to Feedback": [
        "This message is about {} encouraging openness to feedback, honesty, listening, accepting critique, being receptive to guidance, or engaging openly in discussion.",
        "El mensaje trata sobre {} relacionado con escuchar activamente, aceptar retroalimentación y estar abierto a sugerencias.",
        "النص يتحدث عن {} فيما يخص تقبل الملاحظات والانفتاح على التوجيه."
    ],

    "Time Availability and Commitment": [
        "This message is about {} related to time commitment, availability, scheduling, punctuality, or dedicating time consistently to mentoring.",
        "El mensaje trata sobre {} relacionado con disponibilidad de tiempo, compromiso o horarios.",
        "النص يتحدث عن {} فيما يخص الالتزام الزمني أو المواعيد."
    ],

    "Social Media and Digital Presence": [
        "This message is about {} related specifically to social media accounts, online presence, digital branding, content, followers, or marketing on social platforms.",
        "El mensaje trata sobre {} relacionado con redes sociales, presencia digital o marca online.",
        "النص يتحدث عن {} فيما يخص وسائل التواصل الاجتماعي أو el existencia digital."
    ],

    "Understanding Mentor Background": [
        "This message is about {} where the mentee is asked to learn about the mentor’s background, experience, expertise, profile, biography, or what the mentor does, not about the mentee’s business.",
        "El mensaje trata sobre {} relacionado con conocer la experiencia, trayectoria o perfil del mentor, no del negocio del emprendedor.",
        "النص يتحدث عن {} فيما يخص التعرف على خلفية أو خبرة المرشد، وليس المشروع التجاري."
    ],

    "No Preparation Needed": [
        "This message explicitly states that {} no preparation, materials, or actions are required before the mentoring session.",
        "El mensaje indica claramente que {} no se necesita preparación previa.",
        "النص يتحدث عن {} لا يتطلب أي تحضير مسبق."
    ]
}




# -------- Helpers --------
def clean_text(s):
    return " ".join(s.strip().split()) if isinstance(s, str) else ""

def is_valid_text(s, min_len=3):
    return isinstance(s, str) and len(s.strip()) >= min_len

def avg_raw_score_for_label(text, label, classifier):
    """Return average RAW probability for a label across its templates."""
    temps = HYPOTHESIS_TEMPLATES.get(label, [f"This message is about {{}}."])
    scs = []
    for temp in temps:
        out = classifier(text, [label], hypothesis_template=temp, multi_label=True)
        # pipeline returns dict for single input or list for batch
        if isinstance(out, dict):
            # some pipeline versions return keys "labels"/"scores"
            scores = out.get("scores", None) or out.get("score", None)
            if isinstance(scores, list) and len(scores) > 0:
                scs.append(float(scores[0]))
            elif isinstance(scores, float):
                scs.append(float(scores))
            else:
                # fallback
                scs.append(0.0)
        else:
            # list response
            # support new and old output formats
            if isinstance(out, list) and len(out) > 0:
                top = out[0]
                # top is dict with 'labels' and 'scores' or 'score'
                if isinstance(top, dict):
                    if 'scores' in top and isinstance(top['scores'], list) and len(top['scores'])>0:
                        scs.append(float(top['scores'][0]))
                    elif 'score' in top:
                        scs.append(float(top['score']))
                    else:
                        scs.append(0.0)
                else:
                    scs.append(0.0)
            else:
                scs.append(0.0)
    return float(np.mean(scs)) if scs else 0.0

# -------- Load classifier --------
print("Loading model:", MODEL_NAME)
classifier = pipeline("zero-shot-classification", model=MODEL_NAME, device=DEVICE)

# -------- Load data and sample unique top-N (random unique non-null sample) --------
df = INPUT_CSV # df = pd.read_csv(INPUT_CSV, low_memory=False) was changed to this line
if COLUMN not in df.columns:
    raise KeyError(f"Column '{COLUMN}' not found. Available columns: {df.columns.tolist()[:30]}")

unique_texts_df = df[[COLUMN]].dropna().drop_duplicates().reset_index(drop=True)
n = min(NUM_UNIQUE, len(unique_texts_df))
if n == 0:
    raise ValueError("No non-null unique texts found in the column.")
unique_texts = unique_texts_df.sample(n=n, random_state=42).reset_index(drop=True)
texts = unique_texts[COLUMN].astype(str).tolist()
print(f"Classifying {len(texts)} unique random texts (capped at {NUM_UNIQUE}).")

# -------- Classify using RAW scores only --------
results = []
for text in tqdm(texts, desc="classifying"):
    t = clean_text(text)
    if not is_valid_text(t):
        results.append({
            "text": text,
            "top_label": "UNKNOWN",
            "top_score": 0.0,
            "final_label": "UNKNOWN",
            "all_raw_scores": {}
        })
        continue

    # compute RAW averaged scores per label (no normalization)
    raw_scores = {lab: avg_raw_score_for_label(t, lab, classifier) for lab in CANDIDATE_LABELS}

    # pick top and second by RAW score
    sorted_pairs = sorted(raw_scores.items(), key=lambda x: x[1], reverse=True)
    top_label, top_score = sorted_pairs[0]
    second_label, second_score = (sorted_pairs[1] if len(sorted_pairs) > 1 else (None, 0.0))

    # -------- MULTI-LABEL DECISION --------
    strong_labels = [
        lab for lab, score in raw_scores.items()
        if score >= MULTI_LABEL_T
    ]

    secondary_labels = [
        lab for lab, score in raw_scores.items()
        if MULTI_LABEL_T > score >= SECONDARY_T
    ]

    # Limit explosion
    final_labels = strong_labels + secondary_labels
    final_labels = final_labels[:MAX_LABELS]

    if not final_labels:
        final_labels = ["UNKNOWN"]

    primary_label = final_labels[0]

    results.append({
        "text": text,
        "primary_label": primary_label,
        "all_labels": final_labels,
        "num_labels": len(final_labels),
        "all_raw_scores": raw_scores
    })


res_df = pd.DataFrame(results)
res_df.to_csv(OUTPUT_UNIQUE_CSV, index=False)
print("Saved unique classified CSV:", OUTPUT_UNIQUE_CSV)
print("\nSample:")
print(res_df[["text","primary_label","all_labels"]].head(10).to_string(index=False))


Loading model: joeddav/xlm-roberta-large-xnli


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Classifying 57 unique random texts (capped at 100).


classifying: 100%|██████████| 57/57 [00:44<00:00,  1.29it/s]

Saved unique classified CSV: person2_before_we_start__menteedf.csv

Sample:
                                                                                                                                                                                                                                                                                                                                                              text                              primary_label                                                                                                                             all_labels
                                                                                                                                                                                                                                                        Should be committed and passionate about the fields they want to start  and have vision on their business.         Goal Clarity and Direction S

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dfmentor=pd.read_csv('/content/person1_before_we_start__mentordf.csv')
pd.set_option('display.max_colwidth', None)

dfmentor.head(50)

,text,primary_label,all_labels,num_labels,all_raw_scores
0,"I like to work with people with a can-do attitude. My mentors should invest time in pre-work before every session, so we can advance together towards a mutual goal.",Mindset and Motivation,"['Mindset and Motivation', 'Practical Preparation and Readiness', 'Contact Details and Communication Channels']",3,"{'Mindset and Motivation': 0.754262367884318, 'Clarity of Business and Self-Understanding': 0.40968673427899677, 'Practical Preparation and Readiness': 0.9565022786458334, 'Technology and Platform Requirements': 0.5181205732127031, 'Business Information and Supporting Documents': 0.5392119189103445, 'No Preparation Needed': 0.0008774893891920025, 'Time Availability and Commitment': 0.5149781803290049, 'Product Presentation and Portfolio Materials': 0.5237458298603693, 'Contact Details and Communication Channels': 0.7706656058629354, 'Active Listening and Openness to Feedback': 0.8311545054117838, 'Goal Clarity and Direction Setting': 0.7462223569552103, 'Understanding Mentor Background': 0.38547516365845996, 'Social Media and Digital Presence': 0.6141847670078278}"
1,kemauan untuk mendalami hal bisnis yang bersangkutan dalam pariwisata dan juga keuangan,Business Information and Supporting Documents,['Business Information and Supporting Documents'],1,"{'Mindset and Motivation': 0.25998510140925646, 'Clarity of Business and Self-Understanding': 0.5186593756079674, 'Practical Preparation and Readiness': 0.12556064253052077, 'Technology and Platform Requirements': 0.12250852305442095, 'Business Information and Supporting Documents': 0.7216286261876425, 'No Preparation Needed': 0.00048046450441082317, 'Time Availability and Commitment': 0.08854885709782441, 'Product Presentation and Portfolio Materials': 0.36322201291720074, 'Contact Details and Communication Channels': 0.4711202730735143, 'Active Listening and Openness to Feedback': 0.5898332893848419, 'Goal Clarity and Direction Setting': 0.4741111248731613, 'Understanding Mentor Background': 0.0029419094401722154, 'Social Media and Digital Presence': 0.1240580730761091}"
2,"Hal yang perlu disiapkan adalah niat, komitmen, ide yang dapat direalisasikan, dan kemahiran dalam penggunaan teknologi.",Technology and Platform Requirements,"['Technology and Platform Requirements', 'Mindset and Motivation', 'Practical Preparation and Readiness']",3,"{'Mindset and Motivation': 0.7497963507970175, 'Clarity of Business and Self-Understanding': 0.04281103859345118, 'Practical Preparation and Readiness': 0.7466729879379272, 'Technology and Platform Requirements': 0.7533813416957855, 'Business Information and Supporting Documents': 0.06480078771710396, 'No Preparation Needed': 7.200138255332907e-05, 'Time Availability and Commitment': 0.09077860228717327, 'Product Presentation and Portfolio Materials': 0.3619656612475713, 'Contact Details and Communication Channels': 0.3264829305311044, 'Active Listening and Openness to Feedback': 0.03154557260374228, 'Goal Clarity and Direction Setting': 0.7402125696341196, 'Understanding Mentor Background': 0.010198246483923867, 'Social Media and Digital Presence': 0.4873950779438019}"
3,Enough time\r\nBe open \r\nOwn the process,Time Availability and Commitment,"['Time Availability and Commitment', 'Clarity of Business and Self-Understanding', 'Practical Preparation and Readiness']",3,"{'Mindset and Motivation': 0.43864427010218304, 'Clarity of Business and Self-Understanding': 0.6092884341875712, 'Practical Preparation and Readiness': 0.7340148091316223, 'Technology and Platform Requirements': 0.19334805694719157, 'Business Information and Supporting Documents': 0.39102573692798615, 'No Preparation Needed': 0.023460987024009228, 'Time Availability and Commitment': 0.8192718227704366, 'Product Presentation and Portfolio Materials': 0.4744212528069814, 'Contact Details and Communication Channels': 0.5545801520347595, 'Active Listening and Openness to Feedback': 0.7074196934700012, 'Goal Clarity and Di

In [ ]:
dfmentee=pd.read_csv('/content/person1_before_we_start__menteedf.csv')

In [ ]:
pd.set_option('display.max_colwidth', None)

dfmentee.head(50)

,text,primary_label,all_labels,num_labels,all_raw_scores,top_label,top_score,final_label
0,An Entrepreneur should have to tell me what should he want to do then I will give him a way to do that.,Active Listening and Openness to Feedback,"['Active Listening and Openness to Feedback', 'Goal Clarity and Direction Setting']",2.0,"{'Mindset and Motivation': 0.2787834294916441, 'Clarity of Business and Self-Understanding': 0.03784506022930145, 'Practical Preparation and Readiness': 0.12100585084408522, 'Technology and Platform Requirements': 0.20789374007532993, 'Business Information and Supporting Documents': 0.2581460525592168, 'No Preparation Needed': 0.08685989988346894, 'Time Availability and Commitment': 0.464661901195844, 'Product Presentation and Portfolio Materials': 0.45804652757942677, 'Contact Details and Communication Channels': 0.5463552474975586, 'Active Listening and Openness to Feedback': 0.713727593421936, 'Goal Clarity and Direction Setting': 0.6747902582089106, 'Understanding Mentor Background': 0.007218549745933463, 'Social Media and Digital Presence': 0.5758457432190577}",NaN,NaN,NaN
1,Keberanian mencoba,Mindset and Motivation,"['Mindset and Motivation', 'Practical Preparation and Readiness', 'Contact Details and Communication Channels']",3.0,"{'Mindset and Motivation': 0.952017068862915, 'Clarity of Business and Self-Understanding': 0.4240465561548869, 'Practical Preparation and Readiness': 0.767720361550649, 'Technology and Platform Requirements': 0.3409096834560235, 'Business Information and Supporting Documents': 0.5212268829345703, 'No Preparation Needed': 0.04635957349091768, 'Time Availability and Commitment': 0.5459499756495158, 'Product Presentation and Portfolio Materials': 0.6883278091748556, 'Contact Details and Communication Channels': 0.7782970468203226, 'Active Listening and Openness to Feedback': 0.8012425502141317, 'Goal Clarity and Direction Setting': 0.8342097600301107, 'Understanding Mentor Background': 0.5149583915869395, 'Social Media and Digital Presence': 0.7108546594778696}",NaN,NaN,NaN
2,لا شئ.. المهم انه يكون، مستعد نفسيا + الاجتهاد,Practical Preparation and Readiness,['Practical Preparation and Readiness'],1.0,"{'Mindset and Motivation': 0.5094086478153864, 'Clarity of Business and Self-Understanding': 0.01539884057516853, 'Practical Preparation and Readiness': 0.7079195181528727, 'Technology and Platform Requirements': 0.0012728359821873407, 'Business Information and Supporting Documents': 0.004718154688210537, 'No Preparation Needed': 0.00022472699978000796, 'Time Availability and Commitment': 0.009609113292147716, 'Product Presentation and Portfolio Materials': 0.01083878225957354, 'Contact Details and Communication Channels': 0.007564970408566296, 'Active Listening and Openness to Feedback': 0.007093675124148528, 'Goal Clarity and Direction Setting': 0.0070611283493538695, 'Understanding Mentor Background': 0.1951626961429914, 'Social Media and Digital Presence': 0.0026548407040536404}",NaN,NaN,NaN
3,"Nada , esta preparado para contar con honestidad su situacion actual. Tener la actitud de aprender de manera activa , que sintamos la necesidad mutua de resolver lo planteado. disponible con interes y compromiso.",Active Listening and Openness to Feedback,"['Active Listening and Openness to Feedback', 'Understanding Mentor Background', 'Mindset and Motivation']",3.0,"{'Mindset and Motivation': 0.6296215876936913, 'Clarity of Business and Self-Understanding': 0.6120697855949402, 'Practical Preparation and Readiness': 0.4485376576582591, 'Technology and Platform Requirements': 0.2204734024902185, 'Business Information and Supporting Documents': 0.33499933282534283, 'No Preparation Needed': 0.7357649902502695, 'Time Availability and Commitment': 0.5924546470244726, 'Product Presentation and Portfolio Materials': 0.12424353137612343, 'Contact Details and Communication Channels': 0.4564569549014171, 'Active Listening and Openness to Feedback': 0.9496261278788248, 'Goal Clarity and Direct

In [39]:
df_person2_mentor=pd.read_csv('/content/person2_before_we_start__mentordf.csv')

In [40]:
pd.set_option('display.max_colwidth', None)

df_person2_mentor.head(50)

,text,primary_label,all_labels,num_labels,all_raw_scores,top_label,top_score,final_label
0,"Antes de embarcarse en una mentoría conmigo, es importante que el emprendedor realice ciertas acciones para aprovechar al máximo esta experiencia:\r\n\r\n\t1.\tDefinir Objetivos Claros: El emprendedor debe tener una comprensión clara de lo que busca lograr con la mentoría. Definir metas y objetivos específicos ayudará a enfocar la colaboración.\r\n\t2.\tIdentificar Desafíos y Preguntas: Es útil que el emprendedor identifique los desafíos y las preguntas específicas que tiene en mente. Esto permitirá abordar áreas de interés de manera efectiva.\r\n\t3.\tInvestigar Mi Experiencia: Será beneficioso que el emprendedor se tome el tiempo para investigar mi experiencia y antecedentes en marketing digital y negocios. Esto proporcionará un contexto útil para la mentoría.\r\n\t4.\tPreparar Preguntas Relevantes: El emprendedor debe preparar una lista de preguntas y temas que le gustaría discutir durante las sesiones de mentoría. Esto ayudará a mantener las conversaciones enfocadas y productivas.\r\n\t5.\tSer Abierto a la Retroalimentación: La disposición a recibir comentarios constructivos es esencial. El emprendedor debe estar preparado para escuchar y considerar nuevas perspectivas y enfoques.\r\n\t6.\tFlexibilidad y Adaptabilidad: La mentoría puede implicar ajustes en las estrategias y enfoques actuales. El emprendedor debe estar dispuesto a ser flexible y adaptarse a las recomendaciones y orientación proporcionadas.\r\n\t7.\tCompromiso y Coherencia: Es importante que el emprendedor esté comprometido con el proceso de mentoría y participe de manera constante en las sesiones programadas.\r\n\t8.\tRegistrar y Tomar Notas: Llevar un registro de las discusiones y tomar notas durante las sesiones de mentoría ayudará al emprendedor a retener información valiosa y seguir las recomendaciones.",Mindset and Motivation,"['Mindset and Motivation', 'Clarity of Business and Self-Understanding', 'Practical Preparation and Readiness']",3.0,"{'Mindset and Motivation': 0.9973201751708984, 'Clarity of Business and Self-Understanding': 0.997209906578064, 'Practical Preparation and Readiness': 0.9965451757113138, 'Technology and Platform Requirements': 0.9971101880073547, 'Business Information and Supporting Documents': 0.9969887137413025, 'No Preparation Needed': 0.9964069326718649, 'Time Availability and Commitment': 0.9967920581499735, 'Product Presentation and Portfolio Materials': 0.9967714746793112, 'Contact Details and Communication Channels': 0.9969347914059957, 'Active Listening and Openness to Feedback': 0.9967342019081116, 'Goal Clarity and Direction Setting': 0.9970239996910095, 'Understanding Mentor Background': 0.9963902433713278, 'Social Media and Digital Presence': 0.9973430236180624}",NaN,NaN,NaN
1,"Estar seguro que quiere emprender, cual es u punto de partida y minimo cual es su punto de llegada, estar dispuesto a llevar una disciplina en las estrategias que podemos crear ambos de acuerdo a las necesidades de su emprendimiento y conocerlas muy bien, sobre todo sinceridad de su situacion actual para poder juntos recorrer este camino",Clarity of Business and Self-Understanding,"['Clarity of Business and Self-Understanding', 'Active Listening and Openness to Feedback', 'Goal Clarity and Direction Setting']",3.0,"{'Mindset and Motivation': 0.533532922466596, 'Clarity of Business and Self-Understanding': 0.8750230272610983, 'Practical Preparation and Readiness': 0.6805869142214457, 'Technology and Platform Requirements': 0.31821154554684955, 'Business Information and Supporting Documents': 0.4897866944471995, 'No Preparation Needed': 0.0002192802494391799, 'Time Availability and Commitment': 0.389779972533385, 'Product Presentation and Portfolio Materials': 0.46025246878465015, 'Contact Details and Communication Channels': 0.4849473337332408, 'Active Listening and Openness to Feedback': 0.88930743932724, 'Goal Clarity and Direction Setting': 0.75495190

In [ ]:
df_person2_mentor[['text','primary_label','all_labels']].head(50)

In [42]:
df_person2_mentee=pd.read_csv('/content/person2_before_we_start__menteedf.csv')
pd.set_option('display.max_colwidth', None)

df_person2_mentee.head(50)

,text,primary_label,all_labels,num_labels,all_raw_scores,top_label,top_score,final_label
0,Should be committed and passionate about the fields they want to start and have vision on their business.,Goal Clarity and Direction Setting,"['Goal Clarity and Direction Setting', 'Mindset and Motivation', 'Clarity of Business and Self-Understanding']",3.0,"{'Mindset and Motivation': 0.6788071716825167, 'Clarity of Business and Self-Understanding': 0.7251483897368113, 'Practical Preparation and Readiness': 0.2058706066260735, 'Technology and Platform Requirements': 0.3039411250501871, 'Business Information and Supporting Documents': 0.3119230419397354, 'No Preparation Needed': 0.0004264326222861807, 'Time Availability and Commitment': 0.20073188841342926, 'Product Presentation and Portfolio Materials': 0.49400512129068375, 'Contact Details and Communication Channels': 0.48426517844200134, 'Active Listening and Openness to Feedback': 0.02637568007533749, 'Goal Clarity and Direction Setting': 0.8280364076296488, 'Understanding Mentor Background': 0.003005759402488669, 'Social Media and Digital Presence': 0.21027766137073436}",NaN,NaN,NaN
1,"1. Tener perfiles habilitados en las principales redes sociales. (Instagram, Facebook, Twitter)\r\n2. Tener al menos entre 8 y 10 horas libres a la semana para hacer proyectos y pequeñas tareas que le serán asignadas para aprender y mejorar en el campo. \r\n3. Haber identificado el tipo de marcas, comercios o productos con los que le interesa trabajar.",Goal Clarity and Direction Setting,"['Goal Clarity and Direction Setting', 'Social Media and Digital Presence']",2.0,"{'Mindset and Motivation': 0.3514755740761757, 'Clarity of Business and Self-Understanding': 0.21550009648005167, 'Practical Preparation and Readiness': 0.2991607869043946, 'Technology and Platform Requirements': 0.5405044704675674, 'Business Information and Supporting Documents': 0.5360302527745565, 'No Preparation Needed': 0.0005063779511450169, 'Time Availability and Commitment': 0.4101948067545891, 'Product Presentation and Portfolio Materials': 0.4922413130601247, 'Contact Details and Communication Channels': 0.5613499283790588, 'Active Listening and Openness to Feedback': 0.5394682188828787, 'Goal Clarity and Direction Setting': 0.6224554578463236, 'Understanding Mentor Background': 0.05900534025082985, 'Social Media and Digital Presence': 0.6920199990272522}",NaN,NaN,NaN
2,mempunyai semangat untuk berusaha,Mindset and Motivation,"['Mindset and Motivation', 'Product Presentation and Portfolio Materials', 'Contact Details and Communication Channels']",3.0,"{'Mindset and Motivation': 0.9794479608535767, 'Clarity of Business and Self-Understanding': 0.47546182076136273, 'Practical Preparation and Readiness': 0.6717295547326406, 'Technology and Platform Requirements': 0.5315805375576019, 'Business Information and Supporting Documents': 0.5523618757724762, 'No Preparation Needed': 0.29055235783259076, 'Time Availability and Commitment': 0.5864347269137701, 'Product Presentation and Portfolio Materials': 0.7754438519477844, 'Contact Details and Communication Channels': 0.8650726874669393, 'Active Listening and Openness to Feedback': 0.8898937503496805, 'Goal Clarity and Direction Setting': 0.9092414577802023, 'Understanding Mentor Background': 0.5569425622622172, 'Social Media and Digital Presence': 0.8302096128463745}",NaN,NaN,NaN
3,"Lápiz y papel para realizar anotaciones, en caso que lo necesite; asi como tambien que tenga una lista de los posibles temas y preguntas que pueda tener acerca de su emprendimiento.",Practical Preparation and Readiness,"['Practical Preparation and Readiness', 'Active Listening and Openness to Feedback']",2.0,"{'Mindset and Motivation': 0.19342836586292833, 'Clarity of Business and Self-Understanding': 0.4497876645376285, 'Practical Preparation and Readiness': 0.7682110369205475, 'Technology and Platform Requirements': 0.18500567136410004, 'Business Information and Supporting Documents': 0.3496761793891589, 